In [1]:
setwd('/Users/alexis/IEHS Dropbox/Rager Lab/Alexis_Payton/Experiments/1. Compartment Analysis/1.3. Cytokine Distribution Analyses/1.3.3. Cytokine Demographics Distribution Analysis/Input')
Output = ('/Users/alexis/IEHS Dropbox/Rager Lab/Alexis_Payton/Experiments/1. Compartment Analysis/1.3. Cytokine Distribution Analyses/1.3.3. Cytokine Demographics Distribution Analysis/Output')
cur_date = '060521'

library(readxl)
library(tidyverse)
library(reshape2)

#reading in file
data_df <- data.frame(read_excel("CytokineData_102920.xlsx", sheet = 2))

#reading in demographics file
demographics_data_df = data.frame(read_excel("SubjectInfo_102920.xlsx", sheet = 2))

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.0.6     ✔ dplyr   1.0.4
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths




Comparing cytokine distributions between non-smokers and Ecig or CS within each stratified demographic variable (while continuing to stratify by compartment). For example, how does the distribution of each cytokine differ between NS and CS in subject with normal BMIs across each compartment?

In [2]:
#dichotomizing variables
for (i in 1:length(demographics_data_df$Race)){
    if(demographics_data_df$Race[i] != 'W'){
        demographics_data_df$Race[i] = 'NW'
    } 
    if(demographics_data_df$Age[i] <= 30){
        demographics_data_df$Age[i] = 'U' #under 30
    } 
    else {
        demographics_data_df$Age[i] = 'O' #over 30
    }
}

for (i in 1:length(demographics_data_df$Race)){
    if (i != 32){ #ignoring NA value
        if(demographics_data_df$BMI[i] < 25){
            demographics_data_df$BMI[i] = 'N' #normal bmi
    }
        else {
            demographics_data_df$BMI[i] = 'O' #overweight
        }
    }
}

#combining data_df and subject info
data_df = full_join(data_df, demographics_data_df) %>%
    select(!Serum_Cotinine)

Joining, by = c("SubjectNo", "Group", "SubjectID")



In [3]:
head(data_df)

,Original_Identifier,Protein,Conc,Conc_pslog2,Compartment,SubjectNo,Group,SubjectID,Race,Ethnicity,Sex,Age,BMI
,<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,E_C_F_002,IFNg,17.642316,4.220509,NELF,1,NS,NS_1,W,NH,F,U,O
2,E_C_F_002,IL10,2.873724,1.953721,NELF,1,NS,NS_1,W,NH,F,U,O
3,E_C_F_002,IL12p70,1.625272,1.392467,NELF,1,NS,NS_1,W,NH,F,U,O
4,E_C_F_002,IL13,36.117692,5.214035,NELF,1,NS,NS_1,W,NH,F,U,O
5,E_C_F_002,IL1b,104.409217,6.719857,NELF,1,NS,NS_1,W,NH,F,U,O
6,E_C_F_002,IL6,21.159536,4.469856,NELF,1,NS,NS_1,W,NH,F,U,O


In [ ]:
#creating dfs for each stratified demographic variable
white_df = subset(data_df, data_df$Race == 'W')
nonwhite_df = subset(data_df, data_df$Race == 'NW')
male_df = subset(data_df, data_df$Sex == 'M')
female_df = subset(data_df, data_df$Sex == 'F')
under30_df = subset(data_df, data_df$Age == 'U')
over30_df = subset(data_df, data_df$Age == 'O')
normalbmi_df = subset(data_df, data_df$BMI == 'N')
overweight_df = subset(data_df, data_df$BMI == 'O')

In [6]:
#initializing dfs to store values
empty_white_df = data.frame()
empty_nonwhite_df = data.frame()
empty_male_df = data.frame()
empty_female_df = data.frame()
empty_under30_df = data.frame()
empty_over30_df = data.frame()
empty_normalbmi_df = data.frame()
empty_overweight_df = data.frame()

wilcoxon_rank_sum_values_V2 = function(dataframe, empty_df){
    #"""
    #Running wilcoxon rank sum tests after filtering for compartment, protein, and smoking status using a loop. 
    #Ultimately using this test to compare subjects (NS vs. Ecig or CS) within the same demographic.
    
    #:param: subsetted dataframe, empty df
    #:output: a df containing the compartment, comparison, protein, FC, u stat, p value

    #"""
    for (i in 1:length(compartment)){
        for (j in 1:length(smokers)){
            for(k in 1:length(cytokines)){
                NS_df = dataframe %>%
                    filter(Compartment == compartment[i], Protein == cytokines[k], Group == 'NS') %>%
                    select(Compartment, Protein, Conc_pslog2)
                smokers_df = dataframe %>%
                    filter(Compartment == compartment[i], Protein == cytokines[k], Group == smokers[j]) %>%
                    select(Compartment, Protein, Conc_pslog2)

                #running wilcoxon rank sum
                wilcox_test = wilcox.test(NS_df$Conc_pslog2, smokers_df$Conc_pslog2)
                
                #calculating FC to get directionality
                FC = mean(smokers_df$Conc_pslog2)/mean(NS_df$Conc_pslog2)

                #contains smoking status compared, compartment, cytokine, u stat, and p value
                values_df = cbind(compartment[i], smokers[j], cytokines[k], wilcox_test$statistic, FC, 
                              wilcox_test$p.value)
                empty_df = rbind(empty_df, values_df)

            }
        }
    }
    colnames(empty_df) = c("Compartment",'Group', 'Cyotkine', 'Stat', 'FC','P Value')
    
    #adding a column for padj
    PAdj = c()
    for (i in 1:length(compartment)){
        for(j in 1:length(smokers)){
            single_compartment_df = empty_df %>%
                filter(Compartment == compartment[i], Group == smokers[j])
            padj =  p.adjust(as.numeric(as.character(single_compartment_df$`P Value`)), method = "fdr") 
            PAdj = c(PAdj, padj)
        }   
    }
    
    empty_df = cbind(empty_df, PAdj)
    return(empty_df)
}

#calling fn
white_wilcoxon_values = wilcoxon_rank_sum_values_V2(white_df, empty_white_df)
nonwhite_wilcoxon_values = wilcoxon_rank_sum_values_V2(nonwhite_df, empty_nonwhite_df)
male_wilcoxon_values = wilcoxon_rank_sum_values_V2(male_df, empty_male_df)
female_wilcoxon_values = wilcoxon_rank_sum_values_V2(female_df, empty_female_df)
under30_wilcoxon_values = wilcoxon_rank_sum_values_V2(under30_df, empty_under30_df)
over30_wilcoxon_values = wilcoxon_rank_sum_values_V2(over30_df, empty_over30_df)
normalbmi_wilcoxon_values = wilcoxon_rank_sum_values_V2(normalbmi_df, empty_normalbmi_df)
overweight_wilcoxon_values = wilcoxon_rank_sum_values_V2(overweight_df, empty_overweight_df)

Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning 

Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning 

Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning 

Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning 

Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning 

Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning 

Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning 

In [7]:
#creating one final table and exporting
all_wilcoxon_values = rbind(white_wilcoxon_values, nonwhite_wilcoxon_values, male_wilcoxon_values, female_wilcoxon_values, under30_wilcoxon_values, over30_wilcoxon_values,
                  normalbmi_wilcoxon_values, overweight_wilcoxon_values)
write.csv(all_wilcoxon_values, paste0(Output,"/", cur_date, "_Demographic_Distribution_Analysis.csv"), row.names = FALSE)